# **Feature Extraction Methods: Balanced Data With Annotations and Augmentation**

- *Key Features*: [MFCCs, Mel-Spectrograms, Chroma Frequencies, RMS Power]
- *Key Manipulations*: [Varying Window Sizes, Normalization, Average Pooling (Compression), Filtering]
- *Process Assistence*: [Converting them to numpy arrays now, easy label access across features]
- *Conversion*: [To numpy arrays and pkl files]


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time

# Libraries for audio
from IPython.display import Audio
import librosa
import librosa.display

# Training and Testing Split
from sklearn.model_selection import train_test_split

# for normalization & avgpooling features
import tensorflow as tf
# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import random
import IPython.display as ipd

In [13]:
# Variabels to be reused
path =        'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/audio_files' 
npy_path =    'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/' 
test_csv =    'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train-annotated.csv' 
sr = 22050

In [3]:
test_data = pd.read_csv(test_csv)

In [4]:
test_data['species'].value_counts()

species
Dendrocopos minor            378
Alcedo atthis                374
Ixobrychus minutus           374
Acrocephalus arundinaceus    339
Motacilla flava              326
Botaurus stellaris           281
Fulica atra                  276
Charadrius alexandrinus      245
Gallinula chloropus          216
Circus aeruginosus           201
Himantopus himantopus        196
Acrocephalus melanopogon     179
Coracias garrulus            179
Ardea purpurea               164
Porphyrio porphyrio          161
Ciconia ciconia               96
Acrocephalus scirpaceus       90
Tachybaptus ruficollis        89
Anas strepera                 76
Anas platyrhynchos            38
Name: count, dtype: int64

In [5]:
test_data.head()

,Unnamed: 0,species,audio_name,duration,filename_npy,start,end,label
0,0,Ciconia ciconia,XC171088.mp3,32.470204,Ciconia ciconia/XC171088.npy,11.021315,16.283807,clapping
1,1,Ciconia ciconia,XC171088.mp3,32.470204,Ciconia ciconia/XC171088.npy,19.004347,27.850152,clapping
2,2,Ciconia ciconia,XC171088.mp3,32.470204,Ciconia ciconia/XC171088.npy,28.105281,29.315961,clapping
3,3,Alcedo atthis,XC503772.mp3,9.195102,Alcedo atthis/XC503772.npy,1.875801,3.946005,call
4,4,Alcedo atthis,XC503772.mp3,9.195102,Alcedo atthis/XC503772.npy,4.429412,4.707893,call


# **Creating a class to do the extraction**

In [15]:
class Extraction:
  def __init__(self, test_df, window_size, overlap=0.5, npy_path=npy_path, sr=sr, n_mels=128, n_mfcc=20, n_chroma=12, n_cqt=84, hoplength=256, features=['mfcc'], normalize=True, avgpool=False, augment=False):
    """
    Instantiate the Extraction class to extract features.

    Parameters:
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      n_cqt (int): Number of CQT bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'cqt', 'melspectrogram'.
      normalize (bool): Whether to normalize the features.
      avgpool (bool): Whether to avgpool the features.
    """
    
    self.test_df = test_df
    self.npy_path = npy_path
    self.window_size = window_size
    self.overlap = overlap
    self.sr = sr
    self.n_mels = n_mels
    self.n_mfcc = n_mfcc
    self.n_chroma = n_chroma
    self.n_cqt = n_cqt
    self.hoplength = hoplength
    
     # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    self.accepted_feature = ['mfcc', 'chroma', 'cqt', 'melspectrogram']
    for feature in self.features:
      assert feature in self.accepted_feature, f"{feature} is not an accepted feature, only 'mfcc', 'chroma', 'cqt', 'melspectrogram' are accepted features."

    self.normalize = normalize
    self.avgpool = avgpool
    self.augment = augment

    print(f"Test DataFrame shape: {test_df.shape}")

    # extract train and val labels and features
    self.test_y, self.test_features, self.test_ids = self.feature_extraction(self.test_df, window_size=self.window_size)

    # process the features by average pooling
    self.test_features_2D, self.test_features_1D,  = self.process_features(self.test_features)

  def random_augmentation(self, audio):
        """
        Apply random augmentation to the audio: pitch shift, time stretch, or add noise.
        """
        aug_type = np.random.choice(['pitch_up', 'pitch_down', 'noise', 'stretch', 'shrink'])

        if aug_type == 'pitch_up':
            # Random pitch up (between 1 and 4 semitones)
            pitch_shift = np.random.uniform(1, 4)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
        elif aug_type == 'pitch_down':
            # Random pitch down (between -1 and -4 semitones)
            pitch_shift = np.random.uniform(-4, -1)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
        elif aug_type == 'noise':
            # Add random noise at a fraction of the original audio volume
            noise = self.generate_pink_noise(len(audio))
            audio = audio + 0.01 * np.random.uniform(0.2, 0.5) * np.max(audio) * noise
        elif aug_type == 'stretch':
            # Randomly stretch audio (speed up)
            stretch_factor = np.random.uniform(1.1, 1.5)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)
        elif aug_type == 'shrink':
            # Randomly shrink audio (slow down)
            stretch_factor = np.random.uniform(0.8, 0.9)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)

        return audio

  def normalize_audio(self, audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))
  
  def generate_pink_noise(self, num_samples):
    white_noise = np.random.randn(num_samples)
    
    # Apply a filter to convert white noise into pink noise (1/f noise)
    X = np.fft.rfft(white_noise)
    S = np.arange(1, len(X) + 1)  # Frequency scaling
    pink_noise = np.fft.irfft(X / S)

    if len(pink_noise) < num_samples:
        # Pad with zeros if the length is less than num_samples
        pink_noise = np.pad(pink_noise, (0, num_samples - len(pink_noise)), mode='constant')
    elif len(pink_noise) > num_samples:
        # Trim if necessary
        pink_noise = pink_noise[:num_samples]
    
    return self.normalize_audio(pink_noise)
  
  def pad_with_noise(self, audio_data, window_length, window_samples):
    current_length = librosa.get_duration(y=audio_data, sr=self.sr)

    if current_length > window_length:
        return audio_data
    
    target_length_samples = int(window_length * sr) 
    current_length_samples = window_samples
    padding_length_samples = target_length_samples - current_length_samples

    assert target_length_samples == (current_length_samples+padding_length_samples)
    
    # Generate pink noise to pad with
    pink_noise = self.generate_pink_noise(padding_length_samples)
    padded_audio = np.concatenate([audio_data, pink_noise])
    # if len(padded_audio) < target_length_samples:
    #     padded_audio = np.append(padded_audio, self.generate_pink_noise(1))

    assert target_length_samples == len(padded_audio)
    
    return padded_audio
  
  def avg_pooling_keras(self, feature):
    # Clear the previous Keras session
    tf.keras.backend.clear_session()

    # Define the input shape based on features
    input_shape = feature.shape[1:]  # (n_mels, time_steps)

    # Create the Keras model for average pooling
    inputs = tf.keras.layers.Input(shape=input_shape)
    pooled = tf.keras.layers.GlobalAveragePooling1D()(inputs)
    pooling_model = tf.keras.models.Model(inputs=inputs, outputs=pooled)

    # Perform pooling using the model
    pooled_features = pooling_model.predict(feature)

    return pooled_features

#-------------------------Feature Extraction---------------------------------------
  def extract_mfcc(self, window):
    mfcc = librosa.feature.mfcc(y=window, sr=self.sr, n_mfcc=self.n_mfcc, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(mfcc)
    else:
      return mfcc


  def extract_chroma(self, window):
    chroma = librosa.feature.chroma_stft(y=window, sr=self.sr, n_chroma=self.n_chroma, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(chroma)
    else:
      return chroma
   

  def extract_cqt(self, window):
    cqt = librosa.cqt(y=window, sr=sr, hop_length=self.hoplength, n_bins=self.n_cqt)
    cqt_db = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)
    return cqt_db

  def extract_melspectrogram(self, window):
    mel = librosa.feature.melspectrogram(y=window, sr=self.sr, n_mels=self.n_mels, hop_length=self.hoplength)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    if self.normalize:
      return librosa.util.normalize(mel_db)
    else:
      return mel_db
    
  def avgpooling(self, train_X, n_time, n_features):
    """
    Average pooling the train and val features.

    Parameters:
      train_X (npy): Training feature array of shape (batch_size, n_features, n_time)
      val_X (npy): Validation feature array of shape (batch_size, n_features, n_time)
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature array of shape (batch_size, n_features)
      val_X (npy): Avgpooled validation feature array of shape (batch_size, n_features)
    """
    # Clear the Keras session
    tf.keras.backend.clear_session()
    
    # Create the Keras input layer with shape (n_features, n_time)
    input_layer = tf.keras.layers.Input(shape=(n_features, n_time))
    
    # Apply average pooling over the time axis (axis=-1) to reduce n_time
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1))(input_layer)
    
    # Build the model
    pooling_model = tf.keras.models.Model(inputs=input_layer, outputs=avg_pool)

    # Use the model to apply average pooling on the training and validation features
    train_X = pooling_model.predict(train_X)

    return train_X

    
  def process_features(self, train_features_dict):
    train_copy = train_features_dict.copy()
    for each in train_copy.keys():
      
      if each == 'mfcc':
        n_features=self.n_mfcc
      elif each == 'chroma':
        n_features=self.n_chroma
      elif each == 'cqt':
        n_features=self.n_cqt
      elif each == 'melspectrogram':
        n_features=self.n_mels
      
      train_feature = train_copy[each]

      if self.avgpool:
        train_copy[each] = self.avgpooling(train_feature, n_time=train_feature.shape[2], n_features=n_features)
      else:
        train_copy[each] = train_copy[each]
    
    return train_features_dict, train_copy
      

  def feature_extraction(self, dataframe, window_size):
    y = [] # To hold the labels
    ids = []
    features_dict = {item: [] for item in self.features} # Create a key for each feature listed
    print(f"Number of rows in dataframe: {len(dataframe)}")
    for _, row in tqdm(dataframe.iterrows(), desc="Processing data", total=len(dataframe)):
          label = row['species']
          file_path =  self.npy_path + row['filename_npy']  # os.path.join(self.npy_path, row['filename_npy'])
          id = row['audio_name']
          start = row['start']
          end = row['end']

          # print(f"Processing file: {file_path}")

          try:
            
              audio = np.load(file_path)
          except FileNotFoundError:
              print(f"File not found: {file_path}")
              continue


          start = int(start * sr)
          end = int(end * sr)+512



          if end > len(audio):
             end = len(audio)

          sample = audio[start:end]

          if len(sample) < 512:
                continue

          sample = self.normalize_audio(sample)

          sample = self.pad_with_noise(sample, window_length=self.window_size, window_samples=len(sample))
          

          window_samples = int(window_size * self.sr)
          hop_samples = int(window_samples * (1 - self.overlap))  # For overlapping

          # Break the audio into windows with the specified overlap
          audio_windows = librosa.util.frame(sample, frame_length=window_samples, hop_length=hop_samples).T
          
          
          # display(label)
          
          for _, window in enumerate(audio_windows):
              
              y.append(label)
              ids.append(id)

              if len(window) < window_samples:
                  if len(window) < 512*2:
                     continue
                  else:
                      window = self.pad_with_noise(window, window_length=window_size)
              
              # Feature Extraction FR --------------------------------------------------------------------
              # dynatically call the extract_x function to extract the listed features
              for feature in self.features:
                extract = f"extract_{feature}"
                if hasattr(self, extract) and callable(func := getattr(self, extract)):
                  features_dict[feature].append(func(window))

          # cast lists to np arrays
    for each in features_dict.keys():
              features_dict[each] = np.array(features_dict[each])

    y = np.array(y)
    ids = np.array(ids)

          # If not using average pooling, return resized features
    return y, features_dict, ids

## **Window Size = 3s**

In [16]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [17]:
features = Extraction(test_data,
                      window_size=3,
                      features=features_list,
                      avgpool=True
                      )

Test DataFrame shape: (4278, 8)
Number of rows in dataframe: 4278


Processing data:  25%|██▍       | 1063/4278 [00:00<00:00, 10513.28it/s]

File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Ciconia ciconia/XC171088.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Ciconia ciconia/XC171088.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Ciconia ciconia/XC171088.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Alcedo atthis/XC503772.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Alcedo atthis/XC503772.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Alcedo atthis/XC503772.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Alcedo atthis/XC503772.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Alcedo atthis/XC503772.npy
File not found: C:/Users/t

Processing data:  76%|███████▌  | 3254/4278 [00:00<00:00, 10772.45it/s]

File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Coracias garrulus/XC477331.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Coracias garrulus/XC477331.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Coracias garrulus/XC477331.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Coracias garrulus/XC477331.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Coracias garrulus/XC477331.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Coracias garrulus/XC477331.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Coracias garrulus/XC477331.npy
File not found: C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test_audio_npy/Coracias garrulus/XC477331.npy


Processing data: 100%|██████████| 4278/4278 [00:00<00:00, 10747.14it/s]


IndexError: tuple index out of range

In [9]:
test_y = features.test_y

display(test_y.shape)

(9197,)

(1350,)

In [10]:
# Avgpooled Features
test_features_1D = features.test_features_1D
for key in test_features_1D.keys():
  display(key)
  display(test_features_1D[key].shape)

'melspectrogram'

(9197, 128)

'mfcc'

(9197, 20)

'chroma'

(9197, 12)

'cqt'

(9197, 84)

In [12]:
# Not avgpooled Features
test_features_2D = features.test_features_2D
for key in test_features_2D.keys():
  display(key)
  display(test_features_2D[key].shape)

'melspectrogram'

(9197, 128, 259)

'mfcc'

(9197, 20, 259)

'chroma'

(9197, 12, 259)

'cqt'

(9197, 84, 259)

In [14]:
test_ids = features.test_ids

### Encode Classes

In [15]:
label_encoder = LabelEncoder().fit(test_y)
test_y_encoded = label_encoder.transform(test_y)

classes = list(label_encoder.inverse_transform([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]))
print("Encoded classes:", classes)
print("Encoded training labels:", test_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [12 12 12 ...  4  4  4]
Encoded validation labels: [14 14 14 ...  4  4  4]


In [16]:
# Avg Pooled

test_features_1D['label'] = test_y_encoded


test_features_1D['id'] = test_ids


# Not Avg Pooled

test_features_2D['label'] = test_y_encoded

test_features_2D['id'] = test_ids


In [17]:
merged_dict_1D = {'test': test_features_1D}
merged_dict_1D

{'train': {'melspectrogram': array([[-0.08778238, -0.42542818, -0.45237106, ..., -0.9883016 ,
          -0.99088913, -0.9917844 ],
         [-0.08818693, -0.4527624 , -0.48460808, ..., -0.9879749 ,
          -0.9912214 , -0.99346757],
         [-0.06666847, -0.4327347 , -0.45371366, ..., -0.98730004,
          -0.9916132 , -0.9961998 ],
         ...,
         [-0.01765106, -0.9769646 , -0.97906095, ..., -0.86725354,
          -0.8884754 , -0.9976066 ],
         [-0.01777176, -0.7929504 , -0.7817982 , ..., -0.98777676,
          -0.9978452 , -1.        ],
         [-0.03171015, -0.9459309 , -0.8921383 , ..., -0.9837095 ,
          -0.9915168 , -0.9999268 ]], dtype=float32),
  'mfcc': array([[-0.9975469 ,  0.5757494 , -0.13481264, ...,  0.03258222,
           0.01895186,  0.02151072],
         [-0.99573815,  0.5883738 , -0.2239488 , ...,  0.03442121,
           0.02149916,  0.03377577],
         [-0.9786744 ,  0.7425647 , -0.30753863, ...,  0.03714854,
           0.02035247,  0.02320485]

In [18]:
merged_dict_2D = {'test': test_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[[-0.085169  , -0.06403001, -0.06252546, ..., -0.18273787,
           -0.18892042, -0.20181585],
          [-0.25508188, -0.27114728, -0.32115506, ..., -0.38445121,
           -0.39873584, -0.3933129 ],
          [-0.32371444, -0.33958841, -0.38829561, ..., -0.45105063,
           -0.444728  , -0.43501664],
          ...,
          [-0.9979125 , -0.99782803, -0.99764849, ..., -0.99674981,
           -1.        , -0.99783832],
          [-0.99790549, -0.99786567, -0.99781859, ..., -0.99581807,
           -0.99702104, -0.99504838],
          [-1.        , -1.        , -1.        , ..., -1.        ,
           -0.99911522, -0.99625083]],
  
         [[-0.07655078, -0.05783229, -0.05719943, ..., -0.06720283,
           -0.05434324, -0.06117152],
          [-0.22979912, -0.2450997 , -0.29211519, ..., -0.29226506,
           -0.26610162, -0.24624838],
          [-0.28998886, -0.30498022, -0.35125536, ..., -0.38577562,
           -0.34912624, -0.32429723],


### Save the merged dictionary to a pkl

In [19]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/AnnotatedTest/test_features_3s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [20]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/AnnotatedTest/test_features_3s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file